In [ ]:
import wandb
import os

In [ ]:
import os
# os.environ['http_proxy']="http://proxy61.iitd.ac.in:3128"
# os.environ['https_proxy']="http://proxy61.iitd.ac.in:3128"

In [ ]:
from config.read_config_bard import read_config
import torch

# Reading config file
config = read_config("config/config_bard.txt")

In [ ]:
# setting env variables
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['WANDB_NOTEBOOK_NAME'] = 'bard'

In [ ]:
# get the run number
fp = open("run_seq.txt",'r')
read_run_data = fp.readline().strip()

run_data = {}

while(read_run_data):
    key,value = read_run_data.split(" ")
    run_data[key]=int(value)
    read_run_data= fp.readline().strip()

fp.close()

In [ ]:
# new run number
if config['llm_name'] not in run_data.keys():
    run_data[config['llm_name']] = 0

run_number = str(run_data[config['llm_name']]+1)
run_data[config['llm_name']] += 1

fp = open("run_seq.txt",'w')

for i in run_data.keys():
    fp.write(i+" "+str(run_data[i])+"\n")
fp.close()

In [ ]:
model_name = "final_"+config['llm_name']+"_"+config['sql_dataset'].strip().split(".")[0].strip()+"_method_3"
model_tag = config['llm_name']

run = wandb.init(
    project=model_name, 
    notes="Run number "+run_number,
    tags=["openai",model_tag]
)

wandb.config = {"name":"gemini", "Parameters":model_tag}

In [ ]:
prediction_table = wandb.Table(columns=["Query1","Explanation1","Query2","Explanation2","Prompt","Response","F_prompt","F_response"])

In [ ]:
def read_dataset_file(file_loc):
    data_in_file = []

    file_l = file_loc
    fp = open(file_l,'r')
    sentence = fp.readline()
    while sentence:
        data_in_file.append(sentence)
        sentence = fp.readline()
    
    return data_in_file

dataset_file = config['dataset_path'] + config['sql_dataset']
dataset_file_read = read_dataset_file(dataset_file)

samples = read_dataset_file(config['samples_path']+config['sampling_file'])

samples_list = samples[0].strip().split(' ')

for i in range(len(samples_list)):
    samples_list[i] = int(samples_list[i])

In [ ]:
# Reading table create commands

def read_file(file_loc):
    sentence = ""
    file_l = file_loc
    fp = open(file_l,'r')
    sentence = fp.read()
    fp.close()
    return sentence


In [ ]:
# Reading the prompt files
prompt = {}

for i in range(1,int(config['prompt_count'])+1):
    prompt_file_loc = config['prompt_path'] + config['prompt_file_'+str(i)]
    prompt_data = read_file(prompt_file_loc)

    prompt['prompt_file_'+str(i)] = prompt_data

In [ ]:
import google.generativeai as genai

In [ ]:
temperature = 0.2
max_output_tokens = 10000
top_k = 1

In [ ]:
genai.GenerationConfig(
    max_output_tokens= max_output_tokens,
    temperature=temperature,
    top_k=top_k
)

In [ ]:
genai.configure(api_key='')


In [ ]:
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

In [ ]:
print(config['llm_name'])
model = genai.GenerativeModel(config['llm_name'])

In [ ]:
from create_prompt import create_prompt
import time 

prediction_table_temp = []

if not os.path.exists("input_1"):
    os.mkdir("input_1")

if not os.path.exists("input_1/"+model_name):
    os.mkdir("input_1/"+model_name)

if not os.path.exists("input_1/"+model_name+"/"+run_number):
    os.mkdir("input_1/"+model_name+"/"+run_number)

if not os.path.exists("output_1"):
    os.mkdir("output_1")

if not os.path.exists("output_1/"+model_name):
    os.mkdir("output_1/"+model_name)

if not os.path.exists("output_1/"+model_name+"/"+run_number):
    os.mkdir("output_1/"+model_name+"/"+run_number)

start_time = time.time()

for idx, i in enumerate(samples_list):

    q1 = "[SQL_1] "+dataset_file_read[(i)*2].strip()
    if q1[-1]!=";":
        q1+=";"
    
    q2 = "[SQL_2] "+dataset_file_read[(i)*2+1].strip()
    if q2[-1]!=";":
        q2+=";"

    # creating prompt
    message_q1 = create_prompt([prompt['prompt_file_5'],'\n\n', prompt['prompt_file_2'],'\n',q1,'\n\n',prompt['prompt_file_3']],[],-1)
    print(message_q1)
    response_q1 = model.generate_content(message_q1)
    result_q1 = response_q1.text
    print(result_q1)

    message_q2 = create_prompt([prompt['prompt_file_6'],'\n\n', prompt['prompt_file_2'],'\n',q2,'\n\n',prompt['prompt_file_3']],[],-1)
    print(message_q2)
    response_q2 = model.generate_content(message_q2)
    result_q2 = response_q2.text
    print(result_q2)

    # message = create_prompt([tbl_list,'\n',q1,q2,'\n',prompt['prompt_file_1']],create_table,0)
    message = create_prompt([prompt['prompt_file_1'],'\n\n',prompt['prompt_file_2'],'\n',q1,'\n',q2,'\n\n### Explanation for SQL_1\n',result_q1,'\n','\n### Explanation for SQL_2\n',result_q2,'\n\n',prompt['prompt_file_3']],[],-1)
    print(message)

    # message.append({"role":"user", "content":prompt_temp_1})

    response = model.generate_content(message)
    
    try:
        result = response.text
    except:
        print(i)
        result = "Unknown"

    print(result)
    
    fp = open("input_1/"+model_name+"/"+run_number+"/query_"+str(i+1)+".txt",'w')
    fp.write(str(message))
    fp.close()

    fp = open("output_1/"+model_name+"/"+run_number+"/output_"+str(i+1)+".txt",'w')
    fp.write(str(result))
    fp.close()

    prediction_table_temp.append([str(message_q1),str(result_q1),str(message_q2),str(result_q2),str(message),str(result)])
    time.sleep(1)
    
end_time = time.time()
inf_time = end_time - start_time

In [ ]:
import time
from openai import OpenAI

if not os.path.exists("final_result_input"):
    os.mkdir("final_result_input")

if not os.path.exists("final_result_input/"+model_name):
    os.mkdir("final_result_input/"+model_name)

if not os.path.exists("final_result_input/"+model_name+"/"+run_number):
    os.mkdir("final_result_input/"+model_name+"/"+run_number)

if not os.path.exists("final_result_output"):
    os.mkdir("final_result_output")

if not os.path.exists("final_result_output/"+model_name):
    os.mkdir("final_result_output/"+model_name)

if not os.path.exists("final_result_output/"+model_name+"/"+run_number):
    os.mkdir("final_result_output/"+model_name+"/"+run_number)

start_time = time.time()

temperature = 0.2
max_tokens = 1000
frequency_penalty = 0.0

openai_organization = ""
openai_api_key = ""

os.environ['OPENAI_API_KEY'] = openai_api_key
client = OpenAI()

for idx, i in enumerate(samples_list):
    prompt_temp_2 = create_prompt([prompt['prompt_file_4'],'\n',read_file("output_1/"+model_name+"/"+run_number+"/output_"+str(i+1)+".txt"),'\n\n',prompt['prompt_file_3']], [], -1)
    print(prompt_temp_2)

    # message = prompt_temp_2
    # output =  model.generate_content(message)

    message = [{'role':'user', 'content':prompt_temp_2}]

    output =  response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages = message,
        temperature=temperature,
        max_tokens=max_tokens,
        frequency_penalty=frequency_penalty
    )

    try:
        result = output.choices[0].message.content
    except:
        result = "Unknown"
    print(result)

    fp = open("final_result_input/"+model_name+"/"+run_number+"/query_"+str(i+1)+".txt",'w')
    fp.write(prompt_temp_2)
    fp.close()

    fp = open("final_result_output/"+model_name+"/"+run_number+"/output_"+str(i+1)+".txt",'w')
    fp.write(result)
    fp.close()

    prediction_table_temp[idx].append(prompt_temp_2)
    prediction_table_temp[idx].append(result)

    time.sleep(1)

end_time = time.time()
res_time = end_time - start_time
    

In [ ]:
for i in prediction_table_temp:
    prediction_table.add_data(i[0],i[1],i[2],i[3],i[4],i[5],i[6],i[7])

wandb.log({'predictions': prediction_table})
wandb.finish()

In [ ]:
path = "final_result_output/"+model_name+"/"+run_number+"/output_"

In [ ]:
if not os.path.exists("final_output_count"):
     os.mkdir("final_output_count")

if not os.path.exists("final_output_count/"+model_name):
    os.mkdir("final_output_count/"+model_name)
    
if not os.path.exists("final_output_count/"+model_name+"/"+run_number):
    os.mkdir("final_output_count/"+model_name+"/"+run_number)

yes_cnt = 0
no_cnt = 0
no_idea = 0
for idx, i in enumerate(samples_list):
    fp = open(path+str(i+1)+".txt", 'r')
    data = fp.read()

    data = data.lower().replace("  "," ")
    if "non equivalent" in data or "non-equivalent" in data or "not equivalent" in data or "not-equivalent" in data or "nonequivalent" in data or "notequivalent" in data:
        no_cnt+=1
    elif "equivalent" in data:
        yes_cnt+=1
    else:
        print(i+1)
        print(data)
        no_idea +=1

fp = open("final_output_count/"+model_name+"/"+run_number+"/result.txt",'w')
fp.write("yes: "+str(yes_cnt)+"\n")
fp.write("no: "+str(no_cnt)+"\n")
fp.write("no idea: "+str(no_idea)+"\n")
fp.write("Inference time: "+ str(inf_time) +"\n")
fp.write("Result computation time: "+ str(res_time)+ "\n")
fp.write("####### model details ####### \n")
fp.write("model_name: "+str(model_name)+"\n")
fp.write("temperature: "+str(temperature)+"\n")
fp.write("max_output_tokens: "+str(max_output_tokens)+"\n")
fp.write("top_k: "+str(top_k))

fp.close()


In [ ]:
print('yes',yes_cnt)
print('no',no_cnt)
print('no idea',no_idea)